In [2]:
from glob import glob
from gzip import open # NB: overrides standard open()

import numpy as np
import pandas as pd
import pickle as pkl

# y = the regressors, a simple 2D array

y = pd.read_csv('C:/Users/admin/Desktop/курсовая 2.0/y.txt', sep='\t', decimal=',')
# leaving only Tyr, Trp...
y = y.loc[:, ['Tyr', 'Trp']].assign(
	# ...and the sum of all humic concentrations
	humic=y.loc[
		:,['Pahokee peat', 'Nordic aquatic', 'Suwanee river', 'Elliot soil', 'Disser']
	].sum(axis=1)
)
# save the result
y.to_pickle('y.pkl.gz')

# X = the predictors, a tensor
# dimensions: (sample, emission wavelength, excitation wavelength)

# first read the text files individually
# make sure to sort the list of files, they must correspond to rows in y
X_ = [pd.read_csv(f, sep='\t') for f in sorted(glob('C:/Users/admin/Desktop/курсовая 2.0/X — копия/??.txt'))]

# sanity check: wavelengths must be the same in all files
for x in X_[1:]:
	# errors will also happen if any of the files have different dimensions
	if not ((x.axes[0] == X_[0].axes[0]).all() and (x.axes[1] == X_[0].axes[1]).all()):
		# it doesn't quite matter what we raise here
		# the important thing is to complain loudly and crash
		raise Exception(x.axes)

# now create the actual X tensor
X = np.empty((len(X_), *X_[0].shape))
# and fill in the values of each individual spectrum
for i in range(len(X_)):
	X[i,:,:] = X_[i].values

# save the fluorescence intensity values together with wavelengths
# (like Matlab, NumPy doesn't seem to have the concept
# of "named dimensions" in >2-dimensional arrays)
pkl.dump(
	{'X': X, 'emission': X_[0].axes[0].values, 'excitation': X_[1].axes[1].values},
	open('X.pkl.gz', 'wb')
)
print(y)

         Tyr       Trp      humic
0   3.718645  1.658959  45.446791
1   1.519253  2.908843  53.715968
2   3.514326  1.093680  39.837588
3   4.107808  2.413380  31.410666
4   2.397802  2.822168  42.444629
5   1.771192  3.826668  36.479939
6   2.080763  0.700694  36.112553
7   3.959066  3.163625  44.049334
8   1.172333  2.195268  35.354605
9   3.046104  4.011521  47.441840
10  3.345482  1.008305  47.577334
11  2.530967  3.533570  38.622683
12  0.838763  4.215091  44.546831
13  4.261243  4.150726  35.724431
14  2.577719  3.679025  54.640664
15  1.336314  0.847238  49.409746
16  2.164723  4.313643  26.197475
17  3.546828  1.899125  36.226228
18  1.845509  1.326550  43.272738
19  3.942426  1.985984  34.070473
20  1.458561  3.409581  34.193088
21  3.152387  3.504425  33.342049
22  3.836094  3.841496  41.039801
23  2.959377  3.267010  47.996257
24  1.660952  0.743222  42.147702
25  0.978959  2.295310  34.242464
26  0.675008  2.593348  36.909160
27  3.254376  1.214369  28.183879
28  1.975625  